![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png)

# Your First Wherobots Cloud Notebook

Welcome to Wherobots Cloud! This notebook will introduce some basic concepts of working with spatial data in WherobotsDB including:

* Introducing the Wherobots Open Data Catalog
* Querying data with Spatial SQL
* Creating geospatial visualizations in the Wherobots Cloud notebook environment

Along the way we'll introduce some important concepts for working with WherobotsDB like the Spatial DataFrame data structure and querying WherobotsDB databases and tables using Spatial SQL.

Let's get started!

This video provides a short overview of the Wherobots Cloud platform

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('ErkhBuUz-LM', width=560, height=315)

# The Wherobots Notebook Experience

The Wherobots notebook experience is the main developer interface for working with WherobotsDB. This Jupyter environment allows for running existing notebooks in Python or Scala, creating new notebooks, and loading external notebooks including via git version control. You can run each cell individually or select Run >> Run All Cells from the menu to execute the entire notebook.

# Configuring Sedona and Spark

WherobotsDB is a distributed geospatial analytics database engine powered by Apache Sedona and that runs on top of Apache Spark. Wherobots Cloud takes care of managing the Spark cluster so you don't need to think about Spark and can instead focus on your data analysis.

To get started with WherobotsDB we first need to create a `SedonaContext` object. The `SedonaContext` can contain optional configuration for defining what cloud object stores or other data sources our environment is able to access. Your organization's catalogs and the [Wherobots Open Data catalogs](https://docs.wherobots.com/latest/tutorials/opendata/introduction/) are always automatically configured so they're readily available in your notebooks and jobs.

In [ ]:
from sedona.spark import *

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

# Working With Data - The Wherobots Open Data Catalog

Wherobots Cloud includes access to the [Wherobots Open Data Catalog](https://docs.wherobots.com/latest/tutorials/opendata/introduction/), a collection of open datasets that have been curated and transformed into the [Havasu spatial table format](https://docs.wherobots.com/latest/references/havasu/introduction/) for fast efficient geospatial processing.

Community (free tier) users have access to data from the Overture Maps dataset through the `wherobots_open_data` catalog. Upgrading to the Professional Edition includes access to the `wherobots_pro_data` catalog with many additional datasets including weather, wildfire, and surface temperature, US Census, transportation data, and more.

To access the open data tables we can first list all databases in the `wherobots_open_data` catalog:

In [ ]:
sedona.sql("SHOW SCHEMAS IN wherobots_open_data").show()

Next, we can view each table available in the `overture` database:

In [ ]:
sedona.sql("SHOW tables IN wherobots_open_data.overture").show(truncate=False)

Each table has a rich schema as described in the [Overture Maps documentation](https://docs.overturemaps.org/). Here we view the schema of the `places_place` table:

In [ ]:
sedona.table("wherobots_open_data.overture.places_place").printSchema()

# Spatial SQL

We can use Spatial SQL to query data with WherobotsDB. Spatial SQL extends SQL by adding many functions for working with spatial data. These SQL functions begin with either `ST_` for working with vector data or `RS_` for working with raster data. You can find more information about these spatial SQL functions in the [reference documentation.](https://docs.wherobots.com/latest/references/wherobotsdb/vector-data/Overview/)

Let's look at some examples using Spatial SQL to query the Overture data, using the following Spatial SQL functions:

* [`ST_GeomFromWKT`](https://docs.wherobots.com/latest/references/wherobotsdb/vector-data/Constructor/?h=st_geomfromwkt#st_geomfromwkt) - create a geometry from Well Known Text (WKT) format 
* [`ST_DistanceSphere`](https://docs.wherobots.com/latest/references/wherobotsdb/vector-data/Function/?h=st_distancesph#st_distancesphere) - compute the distance between two geometries

Along the way we'll work with different geometry types including points, lines, and polygons.

First we'll create a view called `places` which will essentially create an alias for our `wherobots_open_data.overture.places_place` table.

In [ ]:
sedona.table("wherobots_open_data.overture.places_place").createOrReplaceTempView("places")

Let's start with a simple query to retrieve some example points of interest, including their name, category, and geometry (in this case a point location):

In [ ]:
sedona.sql("SELECT categories.main AS category, names.common[0].value AS name, geometry FROM places LIMIT 10").show(truncate=False)

Let's imagine we're in San Francisco and we want to find all hiking trails nearby. First, we'll use the `ST_GeomFromWKT` SQL function to create a point geometry that represents our current location in San Francisco from longitude, latitude coordinates that we looked up using a GPS device (-122.46552, 37.77196). Then we'll use the `ST_DistanceSphere` function to find all points of interest within a given distance.

In [ ]:
sedona.sql("""
SELECT names.common[0].value AS name, categories.main AS category, geometry
FROM places
WHERE ST_DistanceSphere(ST_GeomFromWKT('POINT (-122.46552 37.77196)'), geometry) < 10000
AND categories.main = 'hiking_trail'
LIMIT 10
""").show(truncate=False)

So far we've just been printing the results of our queries, but we can also save the results to a variable that represents a spatial DataFrame:

In [ ]:
trails_df = sedona.sql("""
SELECT names.common[0].value AS name, categories.main AS category, geometry
FROM places
WHERE ST_DistanceSphere(ST_GeomFromWKT('POINT (-122.46552 37.77196)'), geometry) < 10000
AND categories.main = 'hiking_trail'
""")

# Visualizing Data

There are many options for visualizing geospatial data in Wherobots notebooks, including:

* [SedonaKepler](https://docs.wherobots.com/latest/tutorials/wherobotsdb/vector-data/vector-visualize/?h=sedonakepler#sedonakepler) - an integration with Kepler.gl that allows for visualizing Sedona DataFrames, exploring interactively and configuring styling
* [SedonaPyDeck](https://docs.wherobots.com/latest/tutorials/wherobotsdb/vector-data/vector-visualize/?h=sedonakepler#sedonapydeck) - an integration with Deck.gl that allows for creating geometry visualizations, choropleths, scatterplots, and heatmaps
* Other Python packages can be installed in the notebook environment, allowing you to leverage any tools from the PyData ecosystem


Here we visualize our hiking trails using `SedonaKepler`:

In [ ]:
SedonaKepler.create_map(trails_df, "Hiking Trails")

# Resources

Where to go from here? The following resources will help you as you continue your spatial analytics journey with WherobotsDB and Wherobots Cloud:

## Example Notebooks

* You can find many more example notebooks available in this Jupyter environment in the `notebook_example` directory via the file explorer in the left tab. Specifically:
    * The `wherobots-db` directory includes further examples for working with WherobotsDB with Python, Spatial SQL and the Overture Maps dataset
        - [wherobots-db-example-python.ipynb](./python/wherobots-db/wherobots-db-example-python.ipynb) - loading data from Shapefiles, performing spatial joins, and writing as GeoParquet
        - [wherobots-db-overture-maps.ipynb](./python/wherobots-db/wherobots-db-overture-maps.ipynb) - explore the Overture Maps dataset including points of interest, administrative boundaries, and road networks 
        - [tile-generation-example.ipynb](./python/wherobots-db/tile-generation-example.ipynb) - generate PMTiles vector map tiles for rendering maps using WherobotsDB's scalable vector tiles generator
    * The `havasu` directory contains examples on working with the Havasu spatial table to perform ETL and data analysis using vector and raster data
        - [havasu-iceberg-geometry-etl.ipynb](./python/havasu/havasu-iceberg-geometry-etl.ipynb) - creating Havasu tables, performing spatial opertions, working with spatial indexes to optimize performance
        - [havasu-iceberg-raster-etl.ipynb](./python/havasu/havasu-iceberg-raster-etl.ipynb) - working with the EuroSAT raster dataset as Havasu tables, raster opertions, handling CRS transforms, and benchmarking raster geometry operations
        - [havasu-iceberg-outdb-raster-etl.ipynb](./python/havasu/havasu-iceberg-outdb-raster-etl.ipynb) - demonstrates the out-db method of working with large rasters in WherobotsDB, loading a large GeoTiff and splitting into tiles, joining vector data with rasters
    * The notebooks in the `wherobots-ai` directory show how to make use of WherobotsAI features like raster inference and map matching
        - [gpu/segmentation.ipynb](./python/wherobots-ai/gpu/segmentation.ipynb) - demonstrates Wherobots Query Inference to identify solar farms from satellite imagery
        - [gpu/classification.ipynb](./python/wherobots-ai/gpu/classification.ipynb) - demonstrates Wherobots Query Inference to identify offshore wind farms from satellite imagery
        - [gpu/object_detection.ipynb](./python/wherobots-ai/gpu/object_detection.ipynb) - demonstrates Wherobots Query Inference to classify land cover from satellite imagery
        - [mapmatching_example.ipynb](./python/wherobots-ai/mapmatching_example.ipynb) - matching noisy GPS trajectory data to OpenStreetMap road segments and visualizing the results

*Note: Only 1 notebook can be run at a time. If you want to run another notebook, please shut down the kernel of the current notebook first (See instructions [here](https://jupyterlab.readthedocs.io/en/stable/user/running.html)).*

## Online Resources

* [Wherobots Online Community](https://community.wherobots.com/) - Ask questions, share your projects, explore what others are working on in the community, and connect with other members of the community
* [Wherobots YouTube Channel](https://www.youtube.com/@wherobotsinc.5352) - Find technical tutorials, example videos, and presentations from spatial data experts on the Wherebots YouTube Channel
* [Wherobots Documentation](https://docs.wherobots.com/) - The documentation includes information about how to manage your Wherobots Cloud account, how to work with data using WherobotsDB, as well as reference documentation
* [Wherobots Blog](https://wherobots.com/blogs/) - Keep up to date with the Wherobots and Apache Sedona community including new product announcements, technical tutorials, and highlighting spatial analytics projects